![logo price](./imgs/massp_logo.png)


## Instructors
* Nguyễn Hoàng Phương



# Phần I - Tuần 2:    Bài toán Regression -  xây dựng mô hình dự đoán giá nhà 
![home price](./imgs/home_prices.png)


## Outline
- <a href='#1'>1. Xây dựng mô hình Random Forest trên Cross Validation </a>
    - <a id='#1-1'>1.1. Cross Validation  </a> 
    - <a id='#1-1'>1.2. Feature Engineering  </a> 
    - <a id='#1-1'>1.3. Huấn luyện mô hình Random Forest với Cross Validation  </a>

- <a href='#2'>2. Thuật toán Random Forest Regressor và tối ưu bayes (bayesian optimization)  </a>  
    - <a id='#2-1'>2.1. Thuật toán Random Forest và các tham số quan trọng </a> 
    - <a id='#2-1'>2.2. Tối ưu Bayes (Bayesian optimization) </a> 
    - <a id='#2-1'>3.1. Sử dụng tối ưu bayes để tối ưu tham số thuật toán Random Forest </a>

Nhập các gói thư viện 

In [46]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import sys
import os
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

Đọc dữ liệu 

In [47]:
train = pd.read_csv('./data/house-prices/train.csv')
test  = pd.read_csv('./data/house-prices/test.csv')

In [48]:
train.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


# <a id='1'>1. Xây dựng mô hình Random Forest trên Cross Validation </a>


# <a id='1.1'>1.1. Xây dựng cross validation </a> 

Cross-validation là quy trình phân chia dữ liệu để đánh giá độ chính xác của mô hình trong trường hợp dữ liệu ít

Ví dụ: Chúng ta chia tập dữ liệu ra 5 phần, mỗi phần chiếm 20% dữ liệu

![tut5_crossval](https://i.imgur.com/9k60cVA.png)

Tiến hành huấn luyện sử dụng cùng một loại mô hình.

**Mô hình 1**: Sử dụng phần đầu tiên của dữ liệu làm tập validation thứ 1, 4 phần còn lại làm dữ liệu huấn luyện 

**Mô hình 2**: Sử dụng phần thứ hai của dữ liệu làm tập validation thứ 2, 4 phần còn lại làm dữ liệu huấn luyện, mô hình được sử dụng với cùng một loại tham số với mô hình 1. 

Tiếp tục xây dựng các mô hình. Kết hợp các validation thứ  $i$ của mô hình $i$, chúng ta được tập validation trên toàn bộ $100\%$ dữ liệu. 


**Ưu điểm của việc sử dụng Cross-validation**:

 + Validation trên tập dữ liệu lớn hơn (100% dữ liệu thay vì chỉ 20% dữ liệu) giúp cho thước đo mức độ chính xác của mô hình đảm bảo tin tưởng hơn
 + Tránh được trường hợp overfitting
 + Khi thực hiện dự đoán với tổng hợp của cả 5 mô hình thì độ chính xác cao hơn, vì mô hình được học trên toàn bộ dữ liệu 

**Nhược điểm của việc sử dụng Cross-validation**: 

 + Để đánh giá một loại mô hình cần sử dụng 5 lần huấn luyện (tốn thời gian huấn luyện)
 
**Vậy khi nào nên sử dụng Cross-validation**: 

 + Trong trường hợp dữ liệu ít, mô hình không quá cồng kềnh. Trong trường hợp có nhiều dữ liệu, dữ liệu có tính tổng quát cao, không nhất thiết phải dùng cross-validation.

# <a id='1.2'>1.2. Feature enginering  </a>

Ở phần trước chúng ta đã sủ dụng: 

   + Chúng ta kết hợp cả tập train và test (gọi là data)
   + Feature engineeing trên toàn bộ data. Sau đó chia lại 2 tập riêng dựa vào ID. 
   + Dùng dữ liệu train để xây dựng mô hình, predict trên tập test
   + Sử dụng dữ liệu tập train, phân chia validation để đo đạc độ chính xác của mô hình.

### Tổng quan các bước xử lý data: 

   +  Xóa các hàng có dữ liệu bất thường (là các hàng dữ liệu có diện tích lớn hơn 4000 và giá nhà nhỏ hơn 300.000)
   +  Xử lý dữ liệu thiếu: 
        + Tạo cột dữ liệu mới tương ứng với NaN hoặc không Nan tương ứng với các cột dữ liệu thiếu 
        + Điền NaN bằng các số liệu thích hợp hơn 
   +  Xử lý dữ liệu dạng số : 
        + Log transform với các cột dữ liệu có giá trị lớn
        + Xử lý các cột dữ liệu thời gian (cột năm xây dựng => cột thời gian sử dụng, tính tới năm 2018)
   +  Xử lý dữ liệu dạng phân loại: 
        + One hot encoding

In [49]:
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

### Xóa các cột dữ liệu bất thường ở tập train 

In [50]:
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

### Kết hợp cả tập train và test (gọi là data)

In [51]:
ntrain = train.shape[0]
ntest = test.shape[0]
target = train.SalePrice.values
data = pd.concat((train, test)).reset_index(drop=True)
data.drop(['SalePrice'], axis=1, inplace=True)
print("data size is : {}".format(data.shape))

data size is : (2917, 79)


### Xử lý missing data

In [52]:
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()       
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)       
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)        
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})        
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    return mis_val_table_ren_columns

#### Tạo thêm cột mới xem cột có NaN hay không 

In [53]:
create_nan_columns = ["MiscFeature", "FireplaceQu", "Fence", "FireplaceQu", "LotFrontage"]
for feature in create_nan_columns: 
    data[feature+"_NAN"] = data[feature].isnull()*1

#### Xử lý các trường dữ liệu có số lượng NaN ít 

In [54]:
rate_nan_columns = ['MSZoning', 'BsmtFullBath', 'BsmtHalfBath', 'Functional','Utilities', 
                    'GarageArea','GarageCars','Electrical','KitchenQual','TotalBsmtSF',
                    'BsmtUnfSF', 'BsmtFinSF2',  'BsmtFinSF1','Exterior2nd', 'Exterior1st','SaleType']
for feature in rate_nan_columns:
    data[feature] = data[feature].fillna(data[feature].mode()[0])

#### Chia dữ liệu thiếu thành 2 lọai 

In [55]:
categotical_nan = data[missing_values_table(data).index].select_dtypes(exclude=[np.number])

In [56]:
numeric_nan = data[missing_values_table(data).index].select_dtypes(include=[np.number])

#### Fillnan cho các trường số 

In [57]:
for feature in numeric_nan.columns:
    data[feature]= data[feature].fillna(0)

#### Fillnan cho các trường phân loại 

In [58]:
for feature in categotical_nan.columns:
    data[feature]= data[feature].fillna("None")

#### Kiểm tra kết quả sau khi đã điền dữ liệu thiếu

In [59]:
missing_values_table(data)

,Missing Values,% of Total Values


In [60]:
numeric_features = data.select_dtypes(include=[np.number])
numeric_columns = numeric_features.columns
data[numeric_columns].head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MiscFeature_NAN,FireplaceQu_NAN,Fence_NAN,LotFrontage_NAN
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,0,0,2,2008,1,1,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,0,0,5,2007,1,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,0,0,9,2008,1,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,0,0,2,2006,1,0,1,0
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,0,0,12,2008,1,0,1,0


### Xử lý dữ liệu dạng số

#### Xử lý với dữ liệu năm 

In [61]:
data['YearBuiltDur'] = 2018 - data['YearBuilt']
data['YearRemodAddDur'] = 2018 - data['YearRemodAdd']
data['GarageYrBltDur'] = 2018 - data['GarageYrBlt']

#### Sử dụng phép biến đổi log cho những cột chênh lệch lớn

In [62]:
transform_columns = ["LotArea", "MasVnrArea", "BsmtFinSF1","BsmtUnfSF", "TotalBsmtSF",
                     "1stFlrSF","2ndFlrSF", "GrLivArea","GarageArea", "WoodDeckSF", 
                     "OpenPorchSF","3SsnPorch","ScreenPorch", "PoolArea"]

In [63]:
#data[transform_columns] = data[transform_columns].replace(0, np.nan)
data.loc[:,transform_columns] = np.log1p(data[transform_columns])
#data[transform_columns] = data[transform_columns].replace(np.nan, 0)
data[transform_columns].head(10)

,LotArea,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,3SsnPorch,ScreenPorch,PoolArea
0,9.042040,5.283204,6.561031,5.017280,6.753438,6.753438,6.751101,7.444833,6.308098,0.000000,4.127134,0.000000,0.0,0.0
1,9.169623,0.000000,6.886532,5.652489,7.141245,7.141245,0.000000,7.141245,6.133398,5.700444,0.000000,0.000000,0.0,0.0
2,9.328212,5.093750,6.188264,6.075346,6.825460,6.825460,6.765039,7.488294,6.411818,0.000000,3.761200,0.000000,0.0,0.0
3,9.164401,0.000000,5.379897,6.293419,6.629363,6.869014,6.629363,7.448916,6.466145,0.000000,3.583519,0.000000,0.0,0.0
4,9.565284,5.860786,6.486161,6.196444,7.044033,7.044033,6.960348,7.695758,6.729824,5.262690,4.442651,0.000000,0.0,0.0
5,9.555064,0.000000,6.597146,4.174387,6.680855,6.680855,6.340359,7.217443,6.175867,3.713572,3.433987,5.771441,0.0,0.0
6,9.218804,5.231109,7.222566,5.762051,7.430707,7.435438,0.000000,7.435438,6.456770,5.545177,4.060443,0.000000,0.0,0.0
7,9.247925,5.484797,6.756932,5.379897,7.010312,7.010312,6.891626,7.645398,6.184149,5.463832,5.323010,0.000000,0.0,0.0
8,8.719481,0.000000,0.000000,6.859615,6.859615,6.930495,6.624065,7.481556,6.150603,4.510860,0.000000,0.000000,0.0,0.0
9,8.912069,0.000000,6.747587,4.948760,6.899723,6.982863,0.000000,6.982863,5.327876,0.000000,1.609438,0.000000,0.0,0.0


In [64]:
###  Xử lý dữ liệu dạng phân loại

In [65]:
categorical_features = data.select_dtypes(exclude=[np.number])
categorical_columns = categorical_features.columns
data[categorical_columns].head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
1,RL,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
2,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,None,None,None,WD,Abnorml
4,RL,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal


In [66]:
#### One Hot Encoding

In [67]:
data = pd.get_dummies(data)
print(data.shape)

(2917, 308)


In [68]:
target = train['SalePrice'].values

In [69]:
train_set = data[:ntrain]
test_set = data[ntrain:]

In [70]:
target = np.log1p(train['SalePrice'].values)

# <a id='1.3'>1.3. Xây dựng mô hình Random Forest Regressor với Cross-Validation   </a>  

### Định nghĩa parameter và mô hình RFR trên mỗi fold 

In [71]:
params = {  'bootstrap': True,
            'criterion': 'mse',
            'max_depth': 4,
            'max_features': 'auto',
            'max_leaf_nodes': None,
            'min_impurity_decrease': 0.0,
            'min_impurity_split': None,
            'min_samples_leaf': 1,
            'min_samples_split': 2,
            'min_weight_fraction_leaf': 0.0,
            'n_estimators': 100,
            'n_jobs': 1,
            'oob_score': False,
            'random_state': 42,
            'verbose': 0,
            'warm_start': False}

def run_RFR(X_train, y_train, X_valid, y_valid, params):
    """
    args: 
    + train_X : training dataset
    + valid_X : validataion dataset
    + train_y : target of training set
    + valid_y : target of validation set
    + params : parameters of RandomForestRegressor
    
    output: 
    
    + model   : The RandomForestRegressor model which is trained by the train_X, train_y
    + y_predict : The prediction of valid_X by using the above model
    """
    model = RandomForestRegressor( max_depth = params['max_depth'], random_state=0)
    model = model.fit(X_train, y_train)
    y_predict =  model.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_predict, squared = False)
    print(rmse)
    return  model, y_predict

### Xây dựng tính toán metric trên Cross Validation dựa trên mô hình mỗi single fold 

In [32]:
def RFR_kfold(train_set, target,test_set, params):   
    kf = KFold(n_splits=5, shuffle = True, random_state = 2020) ## định nghĩa k-fold 
    pred_test_full = 0
# Sử dụng loop để huấn luyện mỗi fold
    oof_preds = np.zeros(train_set.shape[0]) ## out of fold prediction 
    for train_index, valid_index in kf.split(train_set):
        X_train , X_valid  = train_set.loc[train_index,:], train_set.loc[valid_index,:]
        y_train, y_valid = target[train_index], target[valid_index]
        model, _ = run_RFR(X_train, y_train, X_valid, y_valid, params) ## get testing prediction and model  
        pred_test_full += model.predict(test_set) ## prediction of testing data
        oof_preds[valid_index] = model.predict(train_set.loc[valid_index,:])  # prediction of out of fold
    print('Full Out-Of-Fold score : %9.6f' 
          % (mean_squared_error(target, oof_preds) ** .5))
    pred_test_full /= 5
    pred_test_full = np.expm1(pred_test_full)
    return pred_test_full, oof_preds, model

### Chạy kết quả 

In [72]:
pred_test_full, oof_preds, model = RFR_kfold(train_set, target,test_set,params)

0.1671231494825477
0.18444753518172774
0.1639618516777441
0.16795476445073826
0.189620966382241
Full Out-Of-Fold score :  0.174922


### ==>> Kết quả RMSLE trên cross-validation đạt: 0.174922

Lưu ý: Kết quả không nói lên sử dụng cross-validation tốt hay xấu hơn sử dụng cách chia train-valid.

# Các thuật toán thường được dùng khi xây dựng mô hình regression 
+ [Linear regression](https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/)
+ [Lasso Regression](https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/)
+ [Ridge Regression](https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/)
+ [Elastic Net Regression](https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/)
+ [Gradient Boosting regression](https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html)
+ [CatBoostRegressor](https://catboost.ai/docs/concepts/python-reference_catboostregressor.html)
+ [Xgboost](https://xgboost.readthedocs.io/en/latest/parameter.html)
+ [Lightgbm](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html)

# <a id='2'>2. Thuật toán Random Forest Regressor và tối ưu bayes </a> 

## <a id='2.1*'>2.1. Giới thiệu tổng quan về Random Forest  </a>
  

### Bagging and Boosting 

<img src="./imgs/bagging-and-boosting.png" alt="drawing" width="500"/>


+ **Bagging** và **Boosting** đều là thuật toán học giám sát dựa vào nguyên lý tập hợp (ensembling): Xây dựng thuật toán cuối cùng ( gọi là strong learner) dựa vào tổng hợp các thuạt toán nhỏ hơn ( gọi là weak learners)

---

+ **Bagging**: các thuật toán con được xây dưng một cách song song không phụ thuộc vào nhau. Dữ liệu huấn luyện của các mô hình con là dữ liệu bất kì trong dữ liệu huấn luyện gốc (lấy một cách bất kì theo cách lấy vật trong túi - **bagging**) 

---


+ **Boosting**: Xây dựng thuật toán mạnh dựa vào chuỗi (sequence) các thuật toán yếu và có cập nhật trọng số (update weight) vào dữ liệu. Mỗi thuật toán con có liên kết với thuật toán trước và sau đó, thúc đẩy độ chính xác hơn (**boosting**).

<img src="./imgs/boosting.png" alt="drawing" width="600"/>

**Hình mô tả nguyên lý thuật toán Bagging** - các tập dữ liệu được lấy một cách độc lập và mô hình con được huấn luyện độc lập với nhau

<img src="./imgs/bagging.png" alt="drawing" width="600"/>

**Hình mô tả nguyên lý thuật toán Boosting** -Các mô hình con có mối quan hệ sequence với nhau, dữ liệu được update trọng số sau mỗi lần huấn luyện tập con 

<img src="./imgs/adaboost.png" alt="drawing" width="500"/>

**Ví dụ điển hình nguyên lý bagging: Random forest** là thuật toán dùng nguyên lý  bagging và sử dụng decision tree ở mỗi thuật toán con (weak learners)

<img src="./imgs/random_foret.png" alt="drawing" width="600"/>

**Ví dụ điển hình nguyên lý boosting : Adaboost** (Adaptive + Boosting) - là thuật toán dùng nguyên lý boosting và sử dụng weak learner là decision tree.

##  Tổng quan Adaboost and Gradient Boosting

+ Giống nhau: Đều dựa vào nguyên lý Boosting 
+ Khác nhau : Cách tạo ra thuật toán con (weak learners) trong các bước lặp 

### Adaboost = Adaptive + Boosting                        


<img src="./imgs/adaboost_final.png" alt="drawing" width="600"/>



#### Adaboost: Mẫu dữ liệu huấn luyện được thay đổi qua từng bước lặp , tăng trọng số ở mẫu đoán sai, giảm trọng số ở mẫu đoán đúng giúp mô hình con tiếp theo học tốt hơn)

#### Adaboost: Mô hình huấn luyện cuối cùng được kết hợp từ những weak learners

### Gradient Boosting = Gradient Descent + Boosting 

Hình mô tả nguyên lý hoạt động của Gradient Boosting 

<img src="./imgs/gradientboosting.png" alt="drawing" width="600"/>

#### Gradient Boosting : Không thay đổi mẫu huấn luyện (không có quá trình update trọng số  của mẫu dữ liệu huấn luyện). 
#### Định nghĩa hàm loss (để tính toán sai lệch), tại mỗi bước lặp sử dụng gradient descent để update mô hình weak learner. 

<img src="./imgs/gradient boosting.png" alt="drawing" width="400"/>

## Các tham số quan trọng của Random Forest


 + max_depth: Random Forest is defined as the longest path between the root node and the leaf node
 + min_sample_split: a parameter that tells the decision tree in a random forest the minimum required number of observations in any given node in order to split it.
 + max_leaf_nodes: This hyperparameter sets a condition on the splitting of the nodes in the tree and hence restricts the growth of the tree.
 + min_samples_leaf: This Random Forest hyperparameter specifies the minimum number of samples that should be present in the leaf node after splitting a node.
 + n_estimators
 + max_sample: The max_samples hyperparameter determines what fraction of the original dataset is given to any individual tree
 + max_features:  This resembles the number of maximum features provided to each tree in a random forest.

In [73]:
## the fucntion for hidden print
class HiddenPrints:
    '''To hide all the printouts'''

    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

## <a id='2.2*'>2.2*. Giới thiệu tổng quan về tối ưu Bayes ([Bayesian Optimization](https://thetalog.com/thetaflow/bayesian-optimization-visualizer/?fbclid=IwAR1soc93sJ-b5nAC5dxxJ95Im3YrA3K3SXTNIoDPzZ3SBag0-jyj4zSHl88))  </a>

#### Tối ưu hóa Bayes là một chiến lược thiết kế tuần tự để tối ưu hóa toàn cục của các hàm số mà chúng ta không hiểu rõ tính chất (black-box-function).

Sử dụng gói thư viện BayesianOptimization

In [74]:
from bayes_opt import BayesianOptimization

### Ví dụ cơ bản gói thư viện BayesianOptimization

Giả sử chúng ta có black box function như ở dưới. 

$$ f(x,y) = -x^2 -(y-1)^2 +1 $$

Ở đây chúng ta hiểu rõ tính chất hàm \(f\). Nhưng ở ví dụ này chúng ta giả sử không biết rõ về hàm \(f\) và chúng ta muốn tìm 

$$Max_{x,y} f(x,y)$$

 với $x\in (2,4)$, và $y \in (-3,3)$

Chúng ta định nghĩa black box function và grid_search

In [75]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1

In [76]:
grid_search = {'x': (2, 4), 'y': (-3, 3)}

### Sử dụng tối ưu Bayes nhờ thư viện BayesianOptimization. Hàm BayesianOptimization gồm 2 tham số quan trọng: 
   +  f: Hàm black-box
   +  pbounds : Không gian cần tìm kiếm 

In [77]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=grid_search,
    random_state=1,
)

#### Để tìm giá trị max của hàm black-box trong khảng giá trị $(2,4)\times (-3,3)$ chúng ta  sử dụng lệnh 

In [78]:
optimizer.maximize(init_points=5,n_iter=5)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -7.135    |  2.834    |  1.322    |
|  2        | -7.78     |  2.0      | -1.186    |
|  3        | -16.13    |  2.294    | -2.446    |
|  4        | -8.341    |  2.373    | -0.9266   |
|  5        | -7.392    |  2.794    |  0.2329   |
|  6        | -15.02    |  4.0      |  0.8624   |
|  7        | -3.799    |  2.184    |  0.83     |
|  8        | -5.343    |  2.0      |  2.531    |
|  9        | -14.29    |  3.36     |  3.0      |
|  10       | -3.363    |  2.0      |  1.603    |


#### Giá trị tốt nhất sử dụng tối ưu bayes đạt: $-3.363$ khi $x=2$, và $y = 1.6$

## <a id='2.2*'>2.2*. Áp dụng tối ưu bayes để tìm kiếm tham số tối ưu cho thuật toán lightgbm  </a>

#### Ở đây chúng ta sẽ áp dụng "giá trị tốt nhất trên cross-validation của thuật toán lightgbm" như hàm  black-box 
#### Grid_seach là  không gian các tham số muốn tối ưu cho thuật toán lightgbm

Cụ thể: 
+ hàm black-box sẽ là hàm số "kfold_model_gbm" ở mục trước 
+ grid_seach sẽ bao gồm 

    + 'max_depth': (4, 8),
    + 'min_samples_leaf': (1,10),
    + 'min_samples_split': (2,6)

In [79]:
def bayes_parameter_opt_RFR(train_set, target, init_round=15, opt_round=25 ,output_process=False):
    # prepare data
    kf = KFold(n_splits=5, shuffle = True, random_state = 2020)
    
    def single_fold_RFE(train_X, train_y, valid_X, valid_y, params): 
        """
        args: 
        + train   : training dataset 
        + train_y : target of training dataset
        + valid   : validation dataset
        + valid_y : target of validation dataset
        +params   : important parameters of RandomForestRegressor model 
        output:
        model after training with training dataset 
        """
        model = RandomForestRegressor( max_depth = params['max_depth'], 
                                       min_samples_leaf =  params['min_samples_leaf'], 
                                       min_samples_split =  params['min_samples_split'], 
                                       n_estimators =  params['n_estimators'], 
                                       random_state=0)
        model = model.fit(train_X, train_y)
        
        return model
    
    grid_search =      {  'max_depth': (4, 8),
                          'min_samples_leaf': (1,10),
                          'min_samples_split': (2,6)}
    
    def kfold_model(max_depth, min_samples_leaf, min_samples_split):
        """
        args: 
        + all parameters we want to search for the RandomForestRegressor model. Here, we want to tune : 
        max_depth, min_samples_leaf, min_samples_split, n_estimators
        
        ouput:
        The value we want to get maximum when we apply baysian optimization. 
        Here we use the MNLSE metric, then the output should be -MNLSE of the validation dataset. 
    
        """
        oof_preds = np.zeros(train_set.shape[0]) ## out of fold prediction 
        ## define parameters
        params = {  'bootstrap': True,
                    'criterion': 'mse',
                    'max_depth': 4,
                    'max_features': 'auto',
                    'max_leaf_nodes': None,
                    'min_impurity_decrease': 0.0,
                    'min_impurity_split': None,
                    'min_samples_leaf': 1,
                    'min_samples_split': 2,
                    'min_weight_fraction_leaf': 0.0,
                    'n_estimators': 100,
                    'n_jobs': 1,
                    'oob_score': False,
                    'random_state': 42,
                    'verbose': 0,
                    'warm_start': False}
        # update parameter search 

        params['max_depth'] = int(round(max_depth))
        params['min_samples_leaf'] = int(round(min_samples_leaf))
        params['min_samples_split'] = int(round(min_samples_split))
        #params['n_estimators'] = int(round(n_estimators))
        
        for train_index, valid_index in kf.split(train_set):
            X_train , X_valid  = train_set.loc[train_index,:], train_set.loc[valid_index,:]
            y_train, y_valid = target[train_index], target[valid_index]
            with HiddenPrints():
                model = single_fold_RFE(X_train, y_train, X_valid, y_valid, params) ## get testing prediction and model  
            oof_preds[valid_index] = model.predict(train_set.loc[valid_index,:])  # prediction of out of fold
        score = -(mean_squared_error(target, oof_preds) ** .5)
        return score
    
    optim = BayesianOptimization(kfold_model, grid_search , random_state=0)
    # optimize
    optim.maximize(init_points=init_round, n_iter=opt_round,acq='ei')
    # output optimization process
    if output_process == True: optim.points_to_csv("./data/bayes_opt_result.csv")

    return optim

In [80]:
optim = bayes_parameter_opt_RFR(train_set, target, init_round=15, opt_round=25 ,output_process=False)

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
|  1        | -0.1523   |  6.195    |  7.437    |  4.411    |
|  2        | -0.1518   |  6.18     |  4.813    |  4.584    |
|  3        | -0.1539   |  5.75     |  9.026    |  5.855    |
|  4        | -0.1529   |  5.534    |  8.126    |  4.116    |
|  5        | -0.1539   |  6.272    |  9.33     |  2.284    |
|  6        | -0.1749   |  4.349    |  1.182    |  5.33     |
|  7        | -0.1508   |  7.113    |  8.83     |  5.914    |
|  8        | -0.1472   |  7.197    |  5.153    |  5.122    |
|  9        | -0.1736   |  4.473    |  6.759    |  2.573    |
|  10       | -0.1457   |  7.779    |  5.697    |  3.659    |
|  11       | -0.1605   |  5.058    |  7.968    |  3.825    |
|  12       | -0.1529   |  6.274    |  1.169    |  4.471    |
|  13       | -0.1523   |  6.448    |  6.552    |  5.775    |
|  14       | -0.1467   |  6.727    |  4.236    |  3.748    |
|  15   

### In ra các các tham số  tối ưu

In [81]:
print(optim.max)

{'target': -0.14353196152988612, 'params': {'max_depth': 7.577470226676313, 'min_samples_leaf': 1.8610942045560186, 'min_samples_split': 3.606457696290453}}


## Kiểm tra lại kết quả

#### Sử dụng lại Kfold và tham số của mô hình tuần 1. 

In [82]:
kf = KFold(n_splits=5, shuffle = True, random_state = 2020)
params = {  'bootstrap': True,
            'criterion': 'mse',
            'max_depth': 4,
            'max_features': 'auto',
            'max_leaf_nodes': None,
            'min_impurity_decrease': 0.0,
            'min_impurity_split': None,
            'min_samples_leaf': 1,
            'min_samples_split': 2,
            'min_weight_fraction_leaf': 0.0,
            'n_estimators': 100,
            'n_jobs': 1,
            'oob_score': False,
            'random_state': 42,
            'verbose': 0,
            'warm_start': False}

#### Update tham số cần tối ưu

In [83]:
params['max_depth'] = int(round(optim.max["params"]["max_depth"]))
params['min_samples_leaf'] = int(round(optim.max["params"]["min_samples_leaf"]))
params['min_samples_split'] = int(round(optim.max["params"]["min_samples_split"]))

#### Chạy lại mô hình dựa trên Kfold và tham số ở trên 

In [84]:
pred_test_full, oof_preds, model = RFR_kfold(train_set, target,test_set,params)

0.1344003574474378
0.15199920983791837
0.13681978788315652
0.14503508669945614
0.15491255684809496
Full Out-Of-Fold score :  0.144851


### Sau khi finetune parameter, kết quả trên tập validation đã có những cải thiện đáng kể

# QA